# 基於 Transformers 的命名實體識別

## Step1 載入套件

In [8]:
!pip install evaluate
!pip install datasets
!pip install transformers[torch]


In [9]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification


In [10]:
# 評估函數
!pip install seqeval -q
evaluate.load("seqeval")

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

## Step2 載入數據集

In [11]:
# 如果可以連路連線，直接使用load_dataset進行下載
ner_datasets = load_dataset("peoples_daily_ner")
# 如果無法連路連線 使用load_from_disk進行離線下載
# from datasets import DatasetDict
# ner_datasets = DatasetDict.load_from_disk("ner_data")
ner_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [12]:
'''
原始數據集的 ner_tags 是沒有 special token的
'''

ner_datasets["train"][0]


{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

### B-I-O (Begin-Inside-Outside) 標記體系

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

O：表示非實體的詞語（Outside）。

B-PER：表示人名實體的開始（Begin）。例如在句子 "李小龍 是 武術 家" 中，“李小龍”的“李”字會被標記為 B-PER。

I-PER：表示位於人名實體內部的詞語（Inside）。在上述例子中，“小”和“龍”會被標記為 I-PER。

B-ORG：表示組織名稱實體的開始。例如在句子 "我 在 联合国 工作" 中，“联合国”的“联”字會被標記為 B-ORG。

I-ORG：表示位於組織名稱實體內部的詞語。在上述例子中，“合”和“国”會被標記為 I-ORG。

B-LOC：表示地點名稱實體的開始。例如在句子 "我 去過 長城" 中，“长城”的“长”字會被標記為 B-LOC。

I-LOC：表示位於地點名稱實體內部的詞語。在上述例子中，“城”會被標記為 I-LOC。

In [13]:
# meta data
ner_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [14]:
# 建立分類項，用於修改模型輸出類別
label_list = ner_datasets["train"].features["ner_tags"].feature.names
label_list



['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## Step3 資料預處理

In [15]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [16]:
tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True)   # 對於 tokenize 的資料，要指定is_split_into_words參數為True

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
tokenizer.decode(tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True)['input_ids'])

'[CLS] 海 钓 比 赛 地 点 在 厦 门 与 金 门 之 间 的 海 域 。 [SEP]'

In [18]:
# 英文字轉換成 token 會被切成 piece
res = tokenizer("interesting word")
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [19]:
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [20]:
ner_datasets["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [21]:
# 借助 word_ids 去 mapping NER 標籤
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    # tokenized_exmaples = {'input_ids': [101, 3862, 7157, 3683, 6612, 1765], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, !]}
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        #  'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                # cross entropy 默認 -100 不計算
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples


In [22]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets


Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [23]:
print(tokenized_datasets["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## Step4 建立模型

In [24]:
# 對於所有非二分類的任務，要記得修改輸出標籤數量，否則會出現 device 錯誤
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

print(f"{label_list}, {len(label_list)}分類")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], 7分類


In [25]:
model.config.num_labels

7

## Step5 創建評估函数

In [26]:
# 可以另外用地端下載評估函數
# seqeval = evaluate.load("seqeval_metric.py")
seqeval = evaluate.load("seqeval")
seqeval



EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [27]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred # batch result [[xxx,xxx], [xxx,xxx], [xxx,xxx]]
    predictions = np.argmax(predictions, axis=-1)

    # 將 id 轉換為原始字串標籤
    # >>> predictions = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
    # >>> references = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    return {
        "f1": result["overall_f1"]
    }



## Step6 撰寫訓練計劃

In [28]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3
)


## Step7 建立訓練器

In [29]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)



## Step8 模型訓練

In [30]:
trainer.train()

  0%|          | 0/981 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.2207, 'learning_rate': 4.745158002038736e-05, 'epoch': 0.15}
{'loss': 0.0474, 'learning_rate': 4.490316004077472e-05, 'epoch': 0.31}
{'loss': 0.0362, 'learning_rate': 4.235474006116208e-05, 'epoch': 0.46}
{'loss': 0.0353, 'learning_rate': 3.980632008154944e-05, 'epoch': 0.61}
{'loss': 0.0308, 'learning_rate': 3.72579001019368e-05, 'epoch': 0.76}
{'loss': 0.029, 'learning_rate': 3.4709480122324164e-05, 'epoch': 0.92}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.020789911970496178, 'eval_f1': 0.9374219508810878, 'eval_runtime': 4.3009, 'eval_samples_per_second': 539.187, 'eval_steps_per_second': 4.418, 'epoch': 1.0}
{'loss': 0.0243, 'learning_rate': 3.2161060142711516e-05, 'epoch': 1.07}
{'loss': 0.0143, 'learning_rate': 2.9612640163098882e-05, 'epoch': 1.22}
{'loss': 0.0153, 'learning_rate': 2.7064220183486238e-05, 'epoch': 1.38}
{'loss': 0.0146, 'learning_rate': 2.45158002038736e-05, 'epoch': 1.53}
{'loss': 0.0144, 'learning_rate': 2.196738022426096e-05, 'epoch': 1.68}
{'loss': 0.0138, 'learning_rate': 1.9418960244648318e-05, 'epoch': 1.83}
{'loss': 0.0118, 'learning_rate': 1.6870540265035677e-05, 'epoch': 1.99}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.017814045771956444, 'eval_f1': 0.9518304431599229, 'eval_runtime': 10.1331, 'eval_samples_per_second': 228.855, 'eval_steps_per_second': 1.875, 'epoch': 2.0}
{'loss': 0.0081, 'learning_rate': 1.4322120285423038e-05, 'epoch': 2.14}
{'loss': 0.0072, 'learning_rate': 1.1773700305810397e-05, 'epoch': 2.29}
{'loss': 0.0075, 'learning_rate': 9.225280326197758e-06, 'epoch': 2.45}
{'loss': 0.0064, 'learning_rate': 6.676860346585118e-06, 'epoch': 2.6}
{'loss': 0.0061, 'learning_rate': 4.128440366972477e-06, 'epoch': 2.75}
{'loss': 0.0057, 'learning_rate': 1.580020387359837e-06, 'epoch': 2.91}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.017478445544838905, 'eval_f1': 0.9565574893791969, 'eval_runtime': 4.2147, 'eval_samples_per_second': 550.212, 'eval_steps_per_second': 4.508, 'epoch': 3.0}
{'train_runtime': 426.0002, 'train_samples_per_second': 146.937, 'train_steps_per_second': 2.303, 'train_loss': 0.02815297569521341, 'epoch': 3.0}


TrainOutput(global_step=981, training_loss=0.02815297569521341, metrics={'train_runtime': 426.0002, 'train_samples_per_second': 146.937, 'train_steps_per_second': 2.303, 'train_loss': 0.02815297569521341, 'epoch': 3.0})

In [31]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.023323791101574898,
 'eval_f1': 0.948319721515598,
 'eval_runtime': 8.2605,
 'eval_samples_per_second': 561.346,
 'eval_steps_per_second': 4.479,
 'epoch': 3.0}

## Step9 模型預測

In [32]:
from transformers import pipeline

In [33]:
# 使用pipeline 進行推理, 要指定 id2label
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  

In [34]:
#　模型是基於GPU訓練要記得指定device
# 對於NER任務, 可以指定aggregation_strategy為simple, 得到具體的結果, 而不是 token 結果

ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")

In [35]:
res = ner_pipe("小明在北京上班")
res

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'PER',
  'score': 0.6713258,
  'word': '小 明',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': 0.9987469,
  'word': '北 京',
  'start': 3,
  'end': 5}]

In [36]:
# 根據 start和end 取實際結果
ner_result = {}
x = "小明在北京上班"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

ner_result

{'PER': ['小明'], 'LOC': ['北京']}